In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from lightgbm import LGBMRegressor
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold,cross_val_score
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("../input/30-days-of-ml/train.csv", index_col='id')
test = pd.read_csv("../input/30-days-of-ml/test.csv", index_col='id')

In [3]:
y = train.target
X = train.drop(['target'], axis=1)

In [4]:
s=(train.dtypes=='object')
object_cols=list(s[s].index)
ordinal_encoder=OrdinalEncoder()
X[object_cols]=ordinal_encoder.fit_transform(X[object_cols])
test[object_cols]=ordinal_encoder.fit_transform(test[object_cols])

In [5]:
lgbm = LGBMRegressor(n_estimators = 140,
                           learning_rate = 0.135,
                           num_leaves = 40,
                           max_depth = 1,
                           min_data_in_leaf = 40,
                           random_state=0)

In [6]:
print(sum(cross_val_score(lgbm, X, y, cv=7, n_jobs=1))/7)

[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
0.028427422110863163


In [7]:
lgbm.fit(X,y)

[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40


LGBMRegressor(learning_rate=0.135, max_depth=1, min_data_in_leaf=40,
              n_estimators=140, num_leaves=40, random_state=0)

In [8]:
predictions=lgbm.predict(test)
output=pd.DataFrame({'Id':test.index,'target': predictions})
output.to_csv('submission.csv',index=False)